In [59]:
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
import json
from collections import defaultdict, Counter
%matplotlib inline

In [62]:
cpts_df = pd.read_csv('preprocessed_data/annotated_cpts_df.csv', index_col=0)

In [63]:
cpts_df['gold']= cpts_df[['a','b','c']].idxmax(axis=1)

In [64]:
cpts_df.head()

,a,b,c,Total,category,hotel_id,fileId,iId,gold
0_0,2,0,1,3,facility,260,0,0,a
1_0,0,0,2,2,pool,260,1,0,c
2_0,2,0,1,3,renovation_room,260,2,0,a
3_0,2,0,1,3,restaurant,260,3,0,a
3_1,2,0,1,3,restaurant,260,3,1,a


In [101]:
# annotated more than twice

cpts_2 = cpts_df[cpts_df['Total']>=2]

In [103]:
cg = defaultdict(list)
for c, g in zip(cpts_2['category'], cpts_2['gold']):
    cg[c].append(g)

In [104]:
cgcounter = {}
for c,g in cg.items():
    cgcounter[c] = (Counter(g), sum(Counter(g).values()))

In [105]:
cgcounter

{'facility': (Counter({'a': 36, 'b': 33, 'c': 13}), 82),
 'pool': (Counter({'c': 15, 'a': 33, 'b': 19}), 67),
 'renovation_room': (Counter({'a': 43, 'b': 7, 'c': 5}), 55),
 'restaurant': (Counter({'a': 43, 'c': 32, 'b': 42}), 117),
 'room': (Counter({'c': 32, 'b': 62, 'a': 54}), 148),
 'reception': (Counter({'b': 36, 'c': 35, 'a': 37}), 108)}

In [108]:
a,b,c =0,0,0
for cat, de in cgcounter.items():
    d , _ = de
    a+=d['a']
    b+=d['b']
    c+=d['c']
    
a,b,c

(246, 199, 132)

In [65]:
cat_gold= defaultdict(list)
for cat, g in zip(cpts_df['category'], cpts_df['gold']):
    cat_gold[cat].append(g)

In [66]:
cat_gold_counter = {}
for cat, l in cat_gold.items():
    cat_gold_counter[cat]= (Counter(l), sum(Counter(l).values()))

In [67]:
a, b , c=0,0,0
for cat, de in cat_gold_counter.items():
    d , _ = de
    a+=d['a']
    b+=d['b']
    c+=d['c']
    
a,b,c

(251, 200, 134)

In [68]:
cpts_per_file={}
for key, group in cpts_df.groupby('fileId'):
    cpts_per_file[key]=len(group)

In [69]:
sum(cpts_per_file.values())

585

In [70]:
np.mean(list(cpts_per_file.values()))

1.5077319587628866

In [71]:
Counter(list(cpts_per_file.values()))

Counter({1: 252, 5: 3, 3: 36, 2: 90, 4: 6, 6: 1})

In [73]:
sents_df = pd.read_csv('preprocessed_data/annotated_sentences_df.csv', index_col=0)
sents_df.head()

,yes-sentiment,yes-aspect,no-sentiment,no-aspect,category,hotel_id,cpt,Total,fileId,iId
0_0,2,2,0,0,facility,260,0,4,0,0
0_2,2,2,0,0,facility,260,0,4,0,2
0_3,2,2,0,0,facility,260,0,4,0,3
0_4,0,2,2,0,facility,260,0,4,0,4
0_5,2,2,0,0,facility,260,0,4,0,5


In [74]:
def gold_aspect(row):
    if row['no-aspect']!=0:
        return False
    else:
        return True

def gold_sentiment(row):
    if row['no-sentiment']!=0:
        return False
    else:
        return True

In [76]:
sents_df['gold_aspect']= sents_df.apply(gold_aspect, axis=1)

In [84]:
sents_df['gold_sentiment']= sents_df.apply(gold_sentiment, axis=1)

In [78]:
sents_df[sents_df['no-sentiment']==1]

,yes-sentiment,yes-aspect,no-sentiment,no-aspect,category,hotel_id,cpt,Total,fileId,iId,gold_aspect,gold_sentiment
1_9,1,2,1,0,pool,260,0,4,1,9,True,False
2_0,1,0,1,2,renovation_room,260,0,4,2,0,False,False
2_1,1,1,1,1,renovation_room,260,0,4,2,1,False,False
2_2,1,2,1,0,renovation_room,260,0,4,2,2,True,False
2_3,1,2,1,0,renovation_room,260,0,4,2,3,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
419_9,1,1,1,1,room,389,0,4,419,9,False,False
419_10,1,0,1,2,room,389,1,4,419,10,False,False
419_13,1,1,1,1,room,389,1,4,419,13,False,False
419_15,1,2,1,0,room,389,1,4,419,15,True,False


In [85]:
Counter(sents_df.category.to_list())

Counter({'facility': 1453,
         'pool': 1231,
         'renovation_room': 1077,
         'restaurant': 1761,
         'room': 2216,
         'reception': 1658})

In [109]:
sents_df.head()

,yes-sentiment,yes-aspect,no-sentiment,no-aspect,category,hotel_id,cpt,Total,fileId,iId,gold_aspect,gold_sentiment
0_0,2,2,0,0,facility,260,0,4,0,0,True,True
0_2,2,2,0,0,facility,260,0,4,0,2,True,True
0_3,2,2,0,0,facility,260,0,4,0,3,True,True
0_4,0,2,2,0,facility,260,0,4,0,4,True,False
0_5,2,2,0,0,facility,260,0,4,0,5,True,True


In [110]:
sents_df['total-sentiment'] = sents_df['yes-sentiment']+sents_df['no-sentiment']
sents_df['total-aspect'] = sents_df['yes-aspect']+sents_df['no-aspect']

In [116]:
sents_aspect = sents_df[sents_df['total-aspect']>=2]

In [117]:
sents_sentiment = sents_df[sents_df['total-sentiment']>=2]

In [121]:
sents_aspect[sents_aspect['gold_sentiment']==True]

,yes-sentiment,yes-aspect,no-sentiment,no-aspect,category,hotel_id,cpt,Total,fileId,iId,gold_aspect,gold_sentiment,total-sentiment,total-aspect
0_0,2,2,0,0,facility,260,0,4,0,0,True,True,2,2
0_2,2,2,0,0,facility,260,0,4,0,2,True,True,2,2
0_3,2,2,0,0,facility,260,0,4,0,3,True,True,2,2
0_5,2,2,0,0,facility,260,0,4,0,5,True,True,2,2
0_6,2,2,0,0,facility,260,0,4,0,6,True,True,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419_20,2,2,0,0,room,389,2,4,419,20,True,True,2,2
419_21,2,2,0,0,room,389,2,4,419,21,True,True,2,2
419_22,2,2,0,0,room,389,2,4,419,22,True,True,2,2
419_24,2,2,0,0,room,389,2,4,419,24,True,True,2,2


In [87]:
Counter(sents_df.gold_aspect)

Counter({True: 6391, False: 3005})

In [88]:
Counter(sents_df.gold_sentiment)

Counter({True: 6043, False: 3353})

In [89]:
d = Counter(sents_df.gold_aspect.to_list())

In [90]:
sum(d.values())

9396

In [91]:
d

Counter({True: 6391, False: 3005})

In [92]:
cat_gold_aspect= defaultdict(list)
for cat, g in zip(sents_df['category'], sents_df['gold_aspect']):
    cat_gold_aspect[cat].append(g)

In [93]:
cat_gold_aspect_counter = {}
for cat, l in cat_gold_aspect.items():
    cat_gold_aspect_counter[cat]= (Counter(l), sum(Counter(l).values()))

In [94]:
cat_gold_aspect_counter

{'facility': (Counter({True: 874, False: 579}), 1453),
 'pool': (Counter({False: 299, True: 932}), 1231),
 'renovation_room': (Counter({False: 95, True: 982}), 1077),
 'restaurant': (Counter({True: 1114, False: 647}), 1761),
 'room': (Counter({False: 675, True: 1541}), 2216),
 'reception': (Counter({False: 710, True: 948}), 1658)}

In [95]:
cat_gold_sentiment = defaultdict(list)
for cat, g in zip(sents_df['category'], sents_df['gold_sentiment']):
    cat_gold_sentiment[cat].append(g)

In [96]:
cat_gold_senti_counter = {}
for cat, l in cat_gold_sentiment.items():
    cat_gold_senti_counter[cat]= (Counter(l), sum(Counter(l).values()))

In [97]:
cat_gold_senti_counter

{'facility': (Counter({True: 992, False: 461}), 1453),
 'pool': (Counter({True: 795, False: 436}), 1231),
 'renovation_room': (Counter({False: 298, True: 779}), 1077),
 'restaurant': (Counter({True: 1057, False: 704}), 1761),
 'room': (Counter({True: 1387, False: 829}), 2216),
 'reception': (Counter({True: 1033, False: 625}), 1658)}